# Data Collection for Actor Data From IMDB

In [2]:
#imports
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [3]:
#adding sections with description data into one df
count = 1
for i in range(1,18):
    if count == 1:
        df = pd.read_csv(f'data/sections/section_{count}.csv')
        count += 1
    else:
        sub_df = pd.read_csv(f'data/sections/section_{count}.csv')
        df = pd.concat([df,sub_df])
        count += 1

In [4]:
df.head()

,Unnamed: 0,id,movie,year,age_rating,imdb_fan_score,rt_critic_score,netflix,hulu,prime_video,disney_plus,directors,genres,country,language,movie_len_mins,rt_movie_link,rt_audience_score,imdb_cast_link
0,0,1,Inception,2010,13+,8.8,87%,1,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0,https://www.rottentomatoes.com/m/inception,0.91,https://www.imdb.com/title/tt1375666/fullcredits
1,1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0,https://www.rottentomatoes.com/m/the_matrix,0.85,https://www.imdb.com/title/tt0133093/fullcredits
2,2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0,https://www.rottentomatoes.com/m/avengers_infi...,0.91,https://www.imdb.com/title/tt4154756/fullcredits
3,3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0,https://www.rottentomatoes.com/m/back_to_the_f...,0.94,https://www.imdb.com/title/tt0088763/fullcredits
4,4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0,https://www.rottentomatoes.com/m/the_good_the_...,0.97,https://www.imdb.com/title/tt0060196/fullcredits


In [5]:
df.drop(columns=['Unnamed: 0'],inplace=True)

In [13]:
#function that takes imdb link and scrapes actor data from each movie site
def get_actors(link):
    actors = []
    try:
        if 'https://www.imdb.com/title/' in link and 'fullcredits' in link:
            res = requests.get(link)
            soup = BeautifulSoup(res.content,'lxml')
            rows = soup.find('table',attrs={'class':'cast_list'}).find_all('tr')

            for row in rows:
                try:
                    actors.append(row.find_all('a')[1].text.strip())
                except:
                    pass

        if len(actors) > 0:
            return actors[:5]
        else:
            return 'no actor data'
    except:
        pass

In [14]:
#collecting actor data
%%time
df['actors'] = df['imdb_cast_link'].apply(get_actors)

CPU times: user 31min 20s, sys: 1min, total: 32min 20s
Wall time: 2h 52min 42s


In [25]:
#saving output to csv
df.to_csv('data/df_with_actor_data.csv')